# Ejercicio: Bosques aleatorios y arquitectura de modelos
En el ejercicio anterior, utilizamos árboles de decisión para predecir si se resolvería un crimen en San Francisco.

Recordemos que los árboles de decisión hicieron un trabajo razonable, pero tienen tendencia a sobreajustarse, lo que significa que los resultados se degradarían considerablemente al utilizar el conjunto de prueba o cualquier dato no visto.

En esta ocasión, utilizaremos bosques aleatorios para corregir esta tendencia.

También veremos cómo la arquitectura del modelo puede influir en su rendimiento.

## Visualización y preparación de los datos
Como de costumbre, vamos a echar otro vistazo rápido al conjunto de datos sobre delincuencia y a dividirlo en conjuntos de entrenamiento y de prueba:

In [2]:
import pandas
#!wget https://raw.githubusercontent.com/MicrosoftDocs/mslearn-introduction-to-machine-learning/main/graphing.py
#!wget https://raw.githubusercontent.com/MicrosoftDocs/mslearn-introduction-to-machine-learning/main/Data/san_fran_crime.csv
import numpy as np
from sklearn.model_selection import train_test_split
import graphing # custom graphing code. See our GitHub repo for details

# Import the data from the .csv file
dataset = pandas.read_csv('san_fran_crime.csv', delimiter="\t")

# Remember to one-hot encode our crime and PdDistrict variables 
categorical_features = ["Category", "PdDistrict"]
dataset = pandas.get_dummies(dataset, columns=categorical_features, drop_first=False)

# Split the dataset in an 90/10 train/test ratio. 
# Recall that our dataset is very large so we can afford to do this
# with only 10% entering the test set
train, test = train_test_split(dataset, test_size=0.1, random_state=2, shuffle=True)

# Let's have a look at the data and the relationship we are going to model
print(dataset.head())
print("train shape:", train.shape)
print("test shape:", test.shape)

   DayOfWeek  Resolution           X          Y  day_of_year  time_in_hours  \
0          5        True -122.403405  37.775421           29      11.000000   
1          5        True -122.403405  37.775421           29      11.000000   
2          1        True -122.388856  37.729981          116      14.983333   
3          2       False -122.412971  37.785788            5      23.833333   
4          5       False -122.419672  37.765050            1       0.500000   

   Category_ARSON  Category_ASSAULT  Category_BAD CHECKS  Category_BRIBERY  \
0               0                 0                    0                 0   
1               0                 0                    0                 0   
2               0                 0                    0                 0   
3               0                 0                    0                 0   
4               0                 0                    0                 0   

   ...  PdDistrict_BAYVIEW  PdDistrict_CENTRAL  PdDistri

¡Espero que esto te resulte familiar! Si no, salte hacia atrás y realice el ejercicio anterior sobre árboles de decisión. 
## Código de evaluación del modelo 
Usaremos el mismo código de evaluación del modelo que usamos en el ejercicio anterior

In [3]:
from sklearn.metrics import balanced_accuracy_score

# Make a utility method that we can re-use throughout this exercise
# To easily fit and test out model

features = [c for c in dataset.columns if c != "Resolution"]

def fit_and_test_model(model):
    '''
    Trains a model and tests it against both train and test sets
    '''  
    global features

    # Train the model
    model.fit(train[features], train.Resolution)

    # Assess its performance
    # -- Train
    predictions = model.predict(train[features])
    train_accuracy = balanced_accuracy_score(train.Resolution, predictions)

    # -- Test
    predictions = model.predict(test[features])
    test_accuracy = balanced_accuracy_score(test.Resolution, predictions)

    return train_accuracy, test_accuracy


print("Ready to go!")

Ready to go!


## Árbol de decisión 
Entrenemos rápidamente un árbol de decisión razonablemente bien ajustado para recordarnos su desempeño:

In [4]:
import sklearn.tree
# re-fit our last decision tree to print out its performance
model = sklearn.tree.DecisionTreeClassifier(random_state=1, max_depth=10) 

dt_train_accuracy, dt_test_accuracy = fit_and_test_model(model)

print("Decision Tree Performance:")
print("Train accuracy", dt_train_accuracy)
print("Test accuracy", dt_test_accuracy)

Decision Tree Performance:
Train accuracy 0.7742407145595661
Test accuracy 0.7597105242913844


## Bosque aleatorio
Un bosque aleatorio es una colección de árboles de decisión que trabajan juntos para calcular la etiqueta de una muestra.

Los árboles de un bosque aleatorio se entrenan de forma independiente, en diferentes particiones de datos, y por tanto desarrollan diferentes sesgos, pero cuando se combinan es menos probable que sobreajusten los datos.

Construyamos un bosque muy sencillo con dos árboles y los parámetros por defecto:

In [5]:
from sklearn.ensemble import RandomForestClassifier

# Create a random forest model with two trees
random_forest = RandomForestClassifier( n_estimators=2,
                                        random_state=2,
                                        verbose=False)

# Train and test the model
train_accuracy, test_accuracy = fit_and_test_model(random_forest)
print("Random Forest Performance:")
print("Train accuracy", train_accuracy)
print("Test accuracy", test_accuracy)

Random Forest Performance:
Train accuracy 0.8842998107846062
Test accuracy 0.734378540999183


Nuestro bosque de dos árboles ha funcionado peor que el de un solo árbol en el conjunto de pruebas, aunque ha hecho un mejor trabajo en el conjunto de trenes.

Hasta cierto punto, esto era de esperar. Los bosques aleatorios suelen funcionar con muchos más árboles. El simple hecho de tener dos le permite sobreajustar los datos de entrenamiento mucho mejor que el árbol de decisión original.

## Modificación del número de árboles
Construyamos varios modelos de bosque, cada uno con un número diferente de árboles, y veamos cómo funcionan:

In [6]:
import graphing

# n_estimators states how many trees to put in the model
# We will make one model for every entry in this list
# and see how well each model performs 
n_estimators = [2, 5, 10, 20, 50]

# Train our models and report their performance
train_accuracies = []
test_accuracies = []

for n_estimator in n_estimators:
    print("Preparing a model with", n_estimator, "trees...")

    # Prepare the model 
    rf = RandomForestClassifier(n_estimators=n_estimator, 
                                random_state=2, 
                                verbose=False)
    
    # Train and test the result
    train_accuracy, test_accuracy = fit_and_test_model(rf)

    # Save the results
    test_accuracies.append(test_accuracy)
    train_accuracies.append(train_accuracy)


# Plot results
graphing.line_2D(dict(Train=train_accuracies, Test=test_accuracies), 
                    n_estimators,
                    label_x="Numer of trees (n_estimators)",
                    label_y="Accuracy",
                    title="Performance X number of trees", show=True)

Preparing a model with 2 trees...
Preparing a model with 5 trees...
Preparing a model with 10 trees...
Preparing a model with 20 trees...
Preparing a model with 50 trees...


Las métricas son buenas para el conjunto de entrenamiento, pero no tanto para el conjunto de prueba. Un mayor número de árboles tiende a ayudar a ambos, pero sólo hasta cierto punto.

Podríamos haber esperado que el número de árboles resolviera nuestro problema de sobreajuste, ¡pero no fue así! Lo más probable es que el modelo sea demasiado complejo en relación con los datos, lo que le permite sobreajustarse al conjunto de entrenamiento.

## Modificación del número mínimo de muestras para el parámetro de división
Recordemos que los árboles de decisión tienen un nodo raíz, nodos internos y nodos hoja, y que los dos primeros pueden dividirse en nodos más nuevos con subconjuntos de datos.

Si dejamos que nuestro modelo se divida y cree demasiados nodos, puede volverse cada vez más complejo y empezar a sobreajustarse.

Una forma de limitar esa complejidad es decirle al modelo que cada nodo debe tener al menos un número determinado de muestras, ya que de lo contrario no podrá dividirse en subnodos.

En otras palabras, podemos establecer el parámetro min_samples_split del modelo en el número mínimo de muestras necesarias para que un nodo pueda dividirse.

Nuestro valor por defecto para min_samples_split es sólo 2, por lo que los modelos se volverán rápidamente demasiado complejos si ese parámetro se deja intacto.

Ahora utilizaremos el modelo con mejor rendimiento anterior, luego lo probaremos con diferentes valores de min_samples_split y compararemos los resultados:

In [7]:
# Shrink the training set temporarily to explore this
# setting with a more normal sample size
full_trainset = train
train = full_trainset[:1000] # limit to 1000 samples

min_samples_split = [2, 10, 20, 50, 100, 500]

# Train our models and report their performance
train_accuracies = []
test_accuracies = []

for min_samples in min_samples_split:
    print("Preparing a model with min_samples_split = ", min_samples)

    # Prepare the model 
    rf = RandomForestClassifier(n_estimators=20,
                                min_samples_split=min_samples,
                                random_state=2, 
                                verbose=False)
    
    # Train and test the result
    train_accuracy, test_accuracy = fit_and_test_model(rf)

    # Save the results
    test_accuracies.append(test_accuracy)
    train_accuracies.append(train_accuracy)


# Plot results
graphing.line_2D(dict(Train=train_accuracies, Test=test_accuracies), 
                    min_samples_split,
                    label_x="Minimum samples split (min_samples_split)",
                    label_y="Accuracy",
                    title="Performance", show=True)

# Rol back the trainset to the full set
train = full_trainset

Preparing a model with min_samples_split =  2
Preparing a model with min_samples_split =  10
Preparing a model with min_samples_split =  20
Preparing a model with min_samples_split =  50
Preparing a model with min_samples_split =  100
Preparing a model with min_samples_split =  500


Como puede verse más arriba, pequeñas restricciones en la complejidad del modelo -limitando su capacidad de dividir nodos- reducen la diferencia entre el rendimiento de entrenamiento y el de prueba. Si esto es sutil, lo hace sin perjudicar en absoluto el rendimiento en las pruebas.

Al limitar la complejidad del modelo, abordamos el sobreajuste, mejorando su capacidad para generalizar y hacer predicciones precisas sobre datos no vistos.

Obsérvese que el uso de min_samples_split=20 nos dio el mejor resultado para el conjunto de pruebas, y que valores más altos empeoraron los resultados.

## Modificación de la profundidad del modelo
Un método relacionado para limitar los árboles es restringir max_depth. Esto es equivalente a max_depth que utilizamos para nuestro árbol de decisión, anteriormente. Su valor por defecto es None, lo que significa que los nodos pueden expandirse hasta que todas las hojas sean puras (todas las muestras tienen la misma etiqueta) o tengan menos muestras que el valor establecido para min_samples_split.

Si max_depth, o min_samples_split es más apropiado depende de la naturaleza de su conjunto de datos, incluyendo su tamaño. Normalmente hay que experimentar para encontrar la mejor configuración. Investiguemos max_depth como si sólo dispusiéramos de 500 muestras de delitos para nuestro conjunto de entrenamiento.

In [8]:
# Shrink the training set temporarily to explore this
# setting with a more normal sample size
full_trainset = train
train = full_trainset[:500] # limit to 500 samples

max_depths = [2, 4, 6, 8, 10, 15, 20, 50, 100]

# Train our models and report their performance
train_accuracies = []
test_accuracies = []

for max_depth in max_depths:
    print("Preparing a model with max_depth = ", max_depth)

    # Prepare the model 
    rf = RandomForestClassifier(n_estimators=20,
                                max_depth=max_depth,
                                random_state=2, 
                                verbose=False)
    
    # Train and test the result
    train_accuracy, test_accuracy = fit_and_test_model(rf)

    # Save the results
    test_accuracies.append(test_accuracy)
    train_accuracies.append(train_accuracy)


# Plot results
graphing.line_2D(dict(Train=train_accuracies, Test=test_accuracies),
                    max_depths,
                    label_x="Maximum depth (max_depths)",
                    label_y="Accuracy",
                    title="Performance", show=True)

# Rol back the trainset to the full set
train = full_trainset

Preparing a model with max_depth =  2
Preparing a model with max_depth =  4
Preparing a model with max_depth =  6
Preparing a model with max_depth =  8
Preparing a model with max_depth =  10
Preparing a model with max_depth =  15
Preparing a model with max_depth =  20
Preparing a model with max_depth =  50
Preparing a model with max_depth =  100


El gráfico anterior nos indica que nuestro modelo se beneficia de un valor más alto de max_depth, hasta el límite de 15. Aumentar la profundidad más allá de este punto empieza a perjudicar el rendimiento de las pruebas, ya que restringe demasiado el modelo para que pueda generalizar.

Aumentar la profundidad más allá de este punto empieza a perjudicar el rendimiento de las pruebas, ya que restringe demasiado el modelo para que pueda generalizar.

Como de costumbre, es importante evaluar distintos valores al establecer los parámetros del modelo y definir su arquitectura.

Un modelo optimizado
Optimizar correctamente un modelo en un conjunto de datos tan grande puede llevar muchas horas, más de las que necesitas dedicar a este ejercicio sólo para aprender. Si desea ejecutar un modelo que ya ha sido optimizado para el conjunto de datos completo, puede ejecutar el código siguiente y comparar su rendimiento con todo lo que hemos visto hasta ahora.

Esto es opcional, pero ten en cuenta que el modelo puede tardar entre 1 y 2 minutos en entrenarse debido a su tamaño y al gran volumen de datos.

In [9]:
# Prepare the model 
rf = RandomForestClassifier(n_estimators=200,
                            max_depth=128,
                            max_features=25,
                            min_samples_split=2,
                            random_state=2, 
                            verbose=False)

# Train and test the result
print("Training model. This may take 1 - 2 minutes")
train_accuracy, test_accuracy = fit_and_test_model(rf)

# Print out results, compared to the decision tree
data = {"Model": ["Decision tree","Final random forest"],
        "Train sensitivity": [dt_train_accuracy, train_accuracy],
        "Test sensitivity": [dt_test_accuracy, test_accuracy]
        }

pandas.DataFrame(data, columns = ["Model", "Train sensitivity", "Test sensitivity"])

Training model. This may take 1 - 2 minutes


,Model,Train sensitivity,Test sensitivity
0,Decision tree,0.774241,0.759711
1,Final random forest,0.999657,0.816087


Como puede verse, el ajuste fino de los parámetros del modelo se tradujo en una mejora significativa de los resultados del conjunto de pruebas.

Resumen
En este ejercicio hemos tratado los siguientes temas

Modelos de bosque aleatorio y en qué se diferencian de los árboles de decisión
Cómo podemos cambiar la arquitectura de un modelo estableciendo diferentes parámetros y cambiando sus valores
La importancia de probar varias combinaciones de parámetros y evaluar estos cambios para mejorar el rendimiento
En el futuro veremos que diferentes modelos tienen arquitecturas en las que se pueden ajustar con precisión los parámetros. Es necesario experimentar para obtener los mejores resultados posibles.